# Roundtable MCP Server: Unified AI Assistant Management

This notebook demonstrates how to use the **Roundtable MCP Server** to manage multiple AI coding assistants through a single, unified interface. Roundtable bridges OpenAI Codex, Claude Code, Cursor, and Gemini through the Model Context Protocol (MCP), eliminating the complexity of managing multiple CLI tools while enabling seamless integration with OpenAI's ecosystem.

## What You'll Learn

- 🚀 **Quick Setup**: Connect to multiple AI assistants with minimal configuration
- 🔧 **Zero-Configuration Intelligence**: Automatic discovery and management of available AI tools
- 🔗 **Unified Interface**: Use the same commands across different AI providers
- 📦 **OpenAI Integration**: Seamlessly combine Roundtable with OpenAI's Responses API
- 🛡️ **Production Ready**: Enterprise-grade reliability and error handling

## Why Roundtable + OpenAI?

**The Challenge**: Modern development teams use multiple AI coding assistants, each with different setup requirements, APIs, and integration complexities. Switching between tools breaks workflow continuity.

**The Solution**: Roundtable MCP Server provides a unified interface to all your AI coding assistants through the standardized MCP protocol, while OpenAI's Responses API provides powerful orchestration and tool calling capabilities.

### Key Benefits:
- **Reduce Integration Complexity**: One MCP interface instead of multiple CLI integrations
- **Eliminate Configuration Overhead**: Automatic tool discovery and availability checking
- **Enable Intelligent Routing**: Let OpenAI models choose the best AI assistant for each task
- **Maintain Session Continuity**: Seamless fallbacks when tools are unavailable
- **Future-Proof Architecture**: MCP standard ensures compatibility with emerging tools

## Installation & Setup

### Prerequisites
- Python 3.8+
- OpenAI API key
- One or more AI CLI tools installed (Codex, Claude Code, Cursor, Gemini)

### Quick Installation

In [ ]:
# Install Roundtable MCP Server
!pip install roundtable-ai

# Install OpenAI client
!pip install openai

### Verify AI Tool Availability

Roundtable automatically detects which AI coding assistants are installed and properly configured on your system.

In [ ]:
# Check which AI assistants are available
!roundtable-ai --check

This command:
- Tests each CLI tool by running `--help` commands
- Saves results to `~/.roundtable/availability_check.json` for caching
- Shows detailed availability report
- Only enables tools that are actually working

## Basic Usage: Roundtable as Standalone MCP Server

First, let's start Roundtable as a standalone MCP server to understand how it works.

In [ ]:
import subprocess
import time
import json
import os
from threading import Thread

# Start Roundtable MCP Server in background
def start_roundtable_server():
    """Start the Roundtable MCP server for demonstration"""
    process = subprocess.Popen(
        ['roundtable-ai', '--agents', 'codex,claude'],  # Enable specific agents
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    return process

# For demonstration - in production, configure this in your MCP client
print("Starting Roundtable MCP Server...")
print("In production, configure this in Claude Desktop, VS Code, or other MCP clients.")
print("See configuration examples below.")

## MCP Client Configuration Examples

### Claude Desktop Configuration
Add to your `~/.config/claude_desktop_config.json`:

```json
{
  "mcpServers": {
    "roundtable-ai": {
      "command": "roundtable-ai",
      "env": {
        "CLI_MCP_SUBAGENTS": "codex,claude,cursor,gemini",
        "CLI_MCP_WORKING_DIR": "/path/to/your/project"
      }
    }
  }
}
```

### VS Code Configuration
Add to your `settings.json`:

```json
{
  "mcp.servers": {
    "roundtable-ai": {
      "command": "roundtable-ai",
      "env": {
        "CLI_MCP_SUBAGENTS": "codex,claude,cursor,gemini"
      }
    }
  }
}
```

### Cursor Configuration
Create or edit `.cursor/mcp.json` in your project root:

```json
{
  "mcpServers": {
    "roundtable-ai": {
      "command": "roundtable-ai",
      "env": {
        "CLI_MCP_SUBAGENTS": "codex,claude,cursor,gemini"
      }
    }
  }
}
```

## Advanced Integration: Roundtable + OpenAI Responses API

The real power of Roundtable comes when integrated with OpenAI's Responses API. This enables intelligent routing between different AI assistants based on task requirements.

In [ ]:
import openai
import os
from typing import List, Dict, Any

# Initialize OpenAI client
client = openai.OpenAI(
    api_key=os.getenv('OPENAI_API_KEY')  # Set your OpenAI API key
)

def create_multi_ai_assistant(available_agents: List[str]) -> Dict[str, Any]:
    """
    Create a configuration for using Roundtable with OpenAI Responses API
    
    Args:
        available_agents: List of AI agents to enable (e.g., ['codex', 'claude', 'gemini'])
    
    Returns:
        Configuration dictionary for OpenAI Responses API
    """
    return {
        "model": "gpt-4.1",
        "tools": [
            {
                "type": "mcp",
                "server_label": "roundtable-ai",
                "server_url": "stdio://roundtable-ai",  # Local stdio transport
                "allowed_tools": [
                    f"execute_{agent}_task" for agent in available_agents
                ] + [
                    f"check_{agent}_availability" for agent in available_agents
                ],
                "require_approval": "never",  # Auto-approve for seamless experience
                "env": {
                    "CLI_MCP_SUBAGENTS": ",".join(available_agents),
                    "CLI_MCP_WORKING_DIR": os.getcwd()
                }
            }
        ]
    }

# Example configuration
config = create_multi_ai_assistant(['codex', 'claude', 'gemini'])
print("Multi-AI Assistant Configuration:")
print(json.dumps(config, indent=2))

## Practical Example: Intelligent Code Generation with Multiple AI Assistants

Let's demonstrate a real-world scenario where OpenAI orchestrates multiple AI coding assistants through Roundtable for optimal results.

In [ ]:
# Simulate a complex coding task that benefits from multiple AI perspectives
coding_prompt = """
I need to implement a Python function that:
1. Reads a CSV file with financial data
2. Calculates moving averages (SMA and EMA)
3. Identifies potential trading signals
4. Exports results to both JSON and XML formats

Please use multiple AI assistants to:
- Codex: Generate the core algorithm and mathematical calculations
- Claude: Review code quality and suggest improvements
- Gemini: Optimize performance and add comprehensive error handling

Compare the different approaches and provide the best solution.
"""

system_prompt = """
You are an intelligent code orchestrator with access to multiple AI coding assistants via Roundtable MCP Server.

When given a coding task:
1. First check which AI assistants are available
2. Route specific aspects of the task to the most suitable assistant:
   - Codex: Complex algorithms, mathematical computations, API integrations
   - Claude: Code review, documentation, best practices, refactoring
   - Gemini: Performance optimization, error handling, testing
   - Cursor: UI/UX code, interactive features, modern frameworks
3. Synthesize the results into a comprehensive solution
4. Highlight the unique contributions from each assistant

Always explain why you chose specific assistants for each task component.
"""

# This would be the actual API call in a real implementation
sample_response_structure = {
    "assistant_routing": {
        "codex": "Core algorithm implementation and mathematical functions",
        "claude": "Code review, documentation, and best practices",
        "gemini": "Performance optimization and error handling"
    },
    "execution_plan": [
        "1. Check availability of all AI assistants",
        "2. Execute core algorithm development with Codex",
        "3. Review and improve code quality with Claude",
        "4. Optimize performance and add error handling with Gemini",
        "5. Synthesize final solution with comprehensive documentation"
    ]
}

print("Intelligent AI Assistant Routing:")
print(json.dumps(sample_response_structure, indent=2))

## Advanced Use Case: Multi-Agent Code Review System

Here's a sophisticated example showing how to create a multi-agent code review system using Roundtable + OpenAI.

In [ ]:
# Multi-agent code review system
def create_code_review_system():
    """
    Creates a comprehensive code review system using multiple AI assistants
    """
    return {
        "model": "gpt-4.1",
        "input": [
            {
                "role": "system",
                "content": [
                    {
                        "type": "input_text",
                        "text": """
You are a code review orchestrator with access to multiple AI coding assistants.

For each code review:
1. **Codex Analysis**: Check algorithmic correctness, logic flow, and implementation efficiency
2. **Claude Review**: Evaluate code quality, readability, documentation, and adherence to best practices  
3. **Gemini Optimization**: Assess performance implications, memory usage, and optimization opportunities
4. **Cursor UX Check**: For frontend code, evaluate user experience and accessibility

Provide a consolidated review with:
- Priority-ranked issues (Critical, High, Medium, Low)
- Specific suggestions from each AI assistant
- Overall code quality score (1-10)
- Recommended next steps
"""
                    }
                ]
            }
        ],
        "tools": [
            {
                "type": "mcp",
                "server_label": "roundtable-ai",
                "server_url": "stdio://roundtable-ai",
                "allowed_tools": [
                    "execute_codex_task",
                    "execute_claude_task", 
                    "execute_gemini_task",
                    "execute_cursor_task",
                    "check_codex_availability",
                    "check_claude_availability",
                    "check_gemini_availability",
                    "check_cursor_availability"
                ],
                "require_approval": "never"
            }
        ],
        "temperature": 0.3,  # Lower temperature for consistent code reviews
        "max_output_tokens": 4096
    }

# Example code to review
sample_code = """
def process_financial_data(file_path):
    import pandas as pd
    data = pd.read_csv(file_path)
    data['sma_20'] = data['close'].rolling(window=20).mean()
    data['ema_12'] = data['close'].ewm(span=12).mean()
    signals = []
    for i in range(len(data)):
        if data['ema_12'].iloc[i] > data['sma_20'].iloc[i]:
            signals.append('BUY')
        else:
            signals.append('SELL')
    data['signals'] = signals
    return data
"""

print("Multi-Agent Code Review System Configuration:")
review_config = create_code_review_system()
print(f"Model: {review_config['model']}")
print(f"Available tools: {len(review_config['tools'][0]['allowed_tools'])}")
print("\nSample code for review:")
print(sample_code)

## Production Deployment Patterns

### Environment Configuration

For production deployments, use environment variables for consistent configuration across different environments.

In [ ]:
# Production environment configuration
production_env_template = """
# Roundtable Configuration
CLI_MCP_SUBAGENTS=codex,claude,gemini,cursor
CLI_MCP_WORKING_DIR=/app/workspace
CLI_MCP_DEBUG=false
CLI_MCP_IGNORE_AVAILABILITY=false

# OpenAI Configuration
OPENAI_API_KEY=your_openai_api_key_here
OPENAI_ORG_ID=your_org_id_here

# AI Assistant API Keys (as required)
ANTHROPIC_API_KEY=your_claude_api_key_here
GOOGLE_API_KEY=your_gemini_api_key_here
"""

# Docker deployment example
dockerfile_content = """
FROM python:3.11-slim

# Install Roundtable and dependencies
RUN pip install roundtable-ai openai

# Install AI CLI tools (as needed)
RUN pip install claude-code-cli
RUN npm install -g @cursor/cli

# Copy application code
COPY . /app
WORKDIR /app

# Start Roundtable MCP Server
CMD ["roundtable-ai"]
"""

print("Production Environment Template:")
print(production_env_template)
print("\nDocker Deployment Example:")
print(dockerfile_content)

### Kubernetes Deployment

For scalable deployments, here's a Kubernetes configuration example:

In [ ]:
kubernetes_config = """
apiVersion: apps/v1
kind: Deployment
metadata:
  name: roundtable-mcp-server
spec:
  replicas: 3
  selector:
    matchLabels:
      app: roundtable-mcp
  template:
    metadata:
      labels:
        app: roundtable-mcp
    spec:
      containers:
      - name: roundtable
        image: roundtable-ai:latest
        ports:
        - containerPort: 8000
        env:
        - name: CLI_MCP_SUBAGENTS
          value: "codex,claude,gemini"
        - name: CLI_MCP_DEBUG
          value: "false"
        - name: OPENAI_API_KEY
          valueFrom:
            secretKeyRef:
              name: ai-secrets
              key: openai-api-key
        resources:
          requests:
            memory: "256Mi"
            cpu: "250m"
          limits:
            memory: "512Mi"
            cpu: "500m"
---
apiVersion: v1
kind: Service
metadata:
  name: roundtable-mcp-service
spec:
  selector:
    app: roundtable-mcp
  ports:
  - protocol: TCP
    port: 80
    targetPort: 8000
  type: LoadBalancer
"""

print("Kubernetes Deployment Configuration:")
print(kubernetes_config)

## Monitoring and Observability

Production deployments need comprehensive monitoring and logging.

In [ ]:
# Monitoring and observability setup
monitoring_config = {
    "health_checks": {
        "availability_endpoint": "/health/availability",
        "performance_endpoint": "/health/performance",
        "dependencies_endpoint": "/health/dependencies"
    },
    "metrics": {
        "request_count": "Total number of AI assistant requests",
        "request_duration": "Average response time per AI assistant",
        "error_rate": "Percentage of failed requests by assistant",
        "availability_score": "Percentage of time each assistant is available"
    },
    "logging": {
        "level": "INFO",
        "format": "json",
        "fields": [
            "timestamp",
            "assistant_type", 
            "request_id",
            "duration_ms",
            "status",
            "error_message"
        ]
    },
    "alerts": {
        "high_error_rate": "Error rate > 5% for any assistant",
        "slow_response": "Average response time > 10 seconds",
        "assistant_unavailable": "Any assistant unavailable for > 5 minutes"
    }
}

print("Monitoring Configuration:")
print(json.dumps(monitoring_config, indent=2))

## Best Practices and Optimization

### Performance Optimization

In [ ]:
# Performance optimization strategies
optimization_strategies = {
    "caching": {
        "availability_cache": "Cache AI assistant availability for 5 minutes",
        "response_cache": "Cache identical requests for 1 hour",
        "session_cache": "Maintain conversation context for 30 minutes"
    },
    "connection_pooling": {
        "max_connections": 10,
        "connection_timeout": 30,
        "keep_alive": True
    },
    "request_batching": {
        "batch_size": 5,
        "batch_timeout": 100,  # milliseconds
        "priority_routing": True
    },
    "intelligent_routing": {
        "load_balancing": "Round-robin with health checks",
        "fallback_strategy": "Automatic failover to available assistants",
        "cost_optimization": "Route to most cost-effective assistant for task type"
    }
}

print("Performance Optimization Strategies:")
print(json.dumps(optimization_strategies, indent=2))

### Security Best Practices

In [ ]:
# Security configuration
security_config = {
    "authentication": {
        "api_key_rotation": "Automatic rotation every 30 days",
        "secure_storage": "Use secrets management (AWS Secrets, Kubernetes Secrets)",
        "access_control": "Role-based access to different AI assistants"
    },
    "data_protection": {
        "encryption_at_rest": "AES-256 encryption for cached data",
        "encryption_in_transit": "TLS 1.3 for all communications",
        "data_retention": "Configurable retention policies"
    },
    "network_security": {
        "firewall_rules": "Restrict access to known IP ranges",
        "rate_limiting": "Per-user and per-assistant rate limits",
        "ddos_protection": "Built-in protection against abuse"
    },
    "audit_logging": {
        "request_logging": "Log all AI assistant interactions",
        "compliance": "SOC 2, GDPR, HIPAA ready",
        "data_lineage": "Track data flow across all assistants"
    }
}

print("Security Configuration:")
print(json.dumps(security_config, indent=2))

## Troubleshooting Guide

### Common Issues and Solutions

In [ ]:
# Troubleshooting guide
troubleshooting_guide = {
    "no_ai_tools_detected": {
        "problem": "Roundtable reports no AI tools available",
        "solutions": [
            "Run 'roundtable-ai --check' to see detailed availability report",
            "Ensure CLI tools are properly installed and in PATH",
            "Check API keys are configured for tools that require them",
            "Use 'CLI_MCP_IGNORE_AVAILABILITY=true' to bypass checking"
        ]
    },
    "mcp_client_not_connecting": {
        "problem": "MCP client cannot connect to Roundtable",
        "solutions": [
            "Verify MCP client configuration matches command aliases",
            "Check working directory is accessible",
            "Enable debug logging with CLI_MCP_DEBUG=true",
            "Ensure roundtable-ai is in PATH"
        ]
    },
    "slow_response_times": {
        "problem": "AI assistant responses are slow",
        "solutions": [
            "Enable connection pooling and caching",
            "Use request batching for multiple operations",
            "Check network connectivity to AI service endpoints",
            "Monitor AI assistant API rate limits"
        ]
    },
    "inconsistent_availability": {
        "problem": "AI assistants randomly become unavailable",
        "solutions": [
            "Check AI assistant service status pages",
            "Verify API key validity and quotas",
            "Implement exponential backoff for retries",
            "Use multiple assistants for redundancy"
        ]
    }
}

def display_troubleshooting_help(issue_key=None):
    """Display troubleshooting information"""
    if issue_key and issue_key in troubleshooting_guide:
        issue = troubleshooting_guide[issue_key]
        print(f"Problem: {issue['problem']}\n")
        print("Solutions:")
        for i, solution in enumerate(issue['solutions'], 1):
            print(f"  {i}. {solution}")
    else:
        print("Available troubleshooting topics:")
        for key, issue in troubleshooting_guide.items():
            print(f"  - {key}: {issue['problem']}")

# Example usage
display_troubleshooting_help("no_ai_tools_detected")

## Testing and Validation

### Integration Tests

In [ ]:
# Integration test suite
def run_integration_tests():
    """Run comprehensive integration tests for Roundtable + OpenAI"""
    
    test_suite = {
        "availability_tests": {
            "test_ai_assistant_detection": "Verify all installed AI assistants are detected",
            "test_availability_caching": "Confirm availability results are cached properly",
            "test_availability_refresh": "Test cache invalidation and refresh"
        },
        "mcp_protocol_tests": {
            "test_tool_listing": "Verify MCP tools/list returns expected tools",
            "test_tool_execution": "Test executing tools with various parameters",
            "test_error_handling": "Confirm graceful error handling for invalid requests"
        },
        "openai_integration_tests": {
            "test_responses_api_integration": "Test OpenAI Responses API with Roundtable MCP",
            "test_multi_assistant_routing": "Verify intelligent routing between assistants",
            "test_session_continuity": "Confirm conversation state is maintained"
        },
        "performance_tests": {
            "test_concurrent_requests": "Handle multiple simultaneous requests",
            "test_response_times": "Verify response times meet SLA requirements",
            "test_memory_usage": "Monitor memory consumption under load"
        },
        "security_tests": {
            "test_api_key_protection": "Ensure API keys are not exposed in logs",
            "test_input_validation": "Verify all inputs are properly validated",
            "test_access_controls": "Test role-based access restrictions"
        }
    }
    
    print("Integration Test Suite:")
    for category, tests in test_suite.items():
        print(f"\n{category.replace('_', ' ').title()}:")
        for test_name, description in tests.items():
            print(f"  ✓ {test_name}: {description}")
    
    return test_suite

# Run the test suite
test_results = run_integration_tests()

# Example test execution command
print("\nTo run tests:")
print("python -m roundtable_mcp_server.test_server")
print("pytest tests/integration/")

## Real-World Use Cases

### Use Case 1: Multi-Assistant Code Generation Pipeline

In [ ]:
# Real-world use case: Code generation pipeline
code_generation_pipeline = """
# Example: Building a REST API with multi-assistant collaboration

# Step 1: Codex generates the API structure
codex_task = '''
Generate a FastAPI REST API with the following endpoints:
- POST /users (create user)
- GET /users/{id} (get user)
- PUT /users/{id} (update user)
- DELETE /users/{id} (delete user)
Include Pydantic models and basic CRUD operations.
'''

# Step 2: Claude reviews and improves code quality
claude_task = '''
Review the generated FastAPI code and improve:
- Add comprehensive error handling
- Implement proper logging
- Add input validation
- Include comprehensive docstrings
- Follow Python best practices
'''

# Step 3: Gemini optimizes performance
gemini_task = '''
Optimize the FastAPI application for production:
- Add database connection pooling
- Implement caching strategies
- Add async/await where appropriate
- Include performance monitoring
- Add comprehensive test suite
'''

# Step 4: Cursor adds frontend integration
cursor_task = '''
Create a React frontend that integrates with the FastAPI backend:
- User management interface
- Form validation
- Error handling and user feedback
- Modern UI with responsive design
'''
"""

print("Multi-Assistant Code Generation Pipeline:")
print(code_generation_pipeline)

### Use Case 2: AI-Powered Code Review Automation

In [ ]:
# Use case: Automated code review system
code_review_automation = {
    "workflow": {
        "trigger": "Pull request created or updated",
        "steps": [
            "1. Roundtable detects available AI assistants",
            "2. OpenAI orchestrates multi-assistant review",
            "3. Each assistant provides specialized feedback", 
            "4. Results are consolidated into unified report",
            "5. Report is posted as PR comment"
        ]
    },
    "assistant_specializations": {
        "codex": [
            "Algorithm correctness",
            "Logic flow analysis", 
            "Performance bottlenecks",
            "Security vulnerabilities"
        ],
        "claude": [
            "Code quality and readability",
            "Best practices adherence",
            "Documentation completeness",
            "Maintainability assessment"
        ],
        "gemini": [
            "Performance optimization opportunities",
            "Memory usage analysis",
            "Scalability considerations",
            "Testing coverage gaps"
        ],
        "cursor": [
            "UI/UX improvements",
            "Accessibility compliance",
            "Frontend performance",
            "User experience flow"
        ]
    },
    "integration_points": {
        "github": "GitHub Actions workflow",
        "gitlab": "GitLab CI/CD pipeline", 
        "bitbucket": "Bitbucket Pipelines",
        "azure_devops": "Azure DevOps pipeline"
    }
}

print("AI-Powered Code Review Automation:")
print(json.dumps(code_review_automation, indent=2))

## Future Roadmap and Extensions

### Planned Enhancements

In [ ]:
# Future roadmap
roadmap = {
    "q1_2025": {
        "smart_routing": "ML-based assistant selection based on task type and performance history",
        "cost_optimization": "Automatic routing to most cost-effective assistant for each task",
        "enhanced_caching": "Intelligent caching with semantic similarity matching"
    },
    "q2_2025": {
        "multi_modal_support": "Support for vision and audio AI assistants",
        "workflow_automation": "Pre-built workflows for common development patterns",
        "analytics_dashboard": "Comprehensive usage analytics and performance insights"
    },
    "q3_2025": {
        "custom_assistants": "Support for custom-trained and fine-tuned models",
        "enterprise_features": "Advanced security, compliance, and governance features",
        "marketplace": "Community marketplace for shared workflows and configurations"
    },
    "q4_2025": {
        "autonomous_agents": "Self-improving agents that learn from usage patterns",
        "multi_cloud_support": "Support for cloud-hosted AI assistant services",
        "advanced_orchestration": "Complex multi-step workflows with conditional logic"
    }
}

print("Roundtable Development Roadmap:")
for quarter, features in roadmap.items():
    print(f"\n{quarter.upper()}:")
    for feature, description in features.items():
        print(f"  • {feature.replace('_', ' ').title()}: {description}")

## Conclusion

The Roundtable MCP Server represents a significant advancement in AI-assisted development, providing:

### Key Benefits Achieved
- **🚀 Simplified Integration**: Single MCP interface for multiple AI coding assistants
- **⚡ Enhanced Productivity**: Intelligent routing and automatic fallbacks
- **🔧 Zero Configuration**: Automatic discovery and management of AI tools
- **🏢 Enterprise Ready**: Production-grade reliability, security, and monitoring
- **🔮 Future Proof**: Built on MCP standard for long-term compatibility

### Perfect Synergy with OpenAI
When combined with OpenAI's Responses API, Roundtable enables:
- **Intelligent Orchestration**: OpenAI models decide which AI assistant to use for each task
- **Seamless Fallbacks**: Automatic switching when assistants are unavailable
- **Optimized Workflows**: Multi-assistant collaboration for complex tasks
- **Consistent Interface**: Same tools work across all development environments

### Getting Started Today
1. **Install**: `pip install roundtable-ai`
2. **Check**: `roundtable-ai --check`
3. **Configure**: Add to your MCP client
4. **Integrate**: Connect with OpenAI Responses API
5. **Scale**: Deploy to production with confidence

### Community and Support
- **Documentation**: [Roundtable AI Documentation](https://askbudi.ai/roundtable)
- **Repository**: [github.com/askbudi/roundtable](https://github.com/askbudi/roundtable)
- **Support**: support@askbudi.ai
- **Community**: Join our Discord for discussions and support

---

*Experience the future of AI-assisted development with Roundtable MCP Server and OpenAI - where multiple AI assistants work together seamlessly through intelligent orchestration.*